## Import Libraries

In [1]:
from os import path
from itertools import product
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter, MaxNLocator
import scipy.io as sio
from brainpipe.classification import *
from brainpipe.system import study
from brainpipe.feature import phase, amplitude, sigfilt
from brainpipe.visual import *
from brainpipe.statistics import *
from scipy.stats import *

## phase Decoding - LowHigh Encoding
### For ALL time points

In [5]:
from brainpipe.feat.utils._feat import _manageWindow
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import StratifiedKFold as SKFold
from sklearn.metrics import roc_auc_score
from numpy.random import permutation

# ============================================================================================
nfreq = 4
nperm = 1000
conds,steps, subjects = ['low','high'],['odor'],['SEMC']#['FERJ','MICP','VACJ','SEMC','LEFC','PIRJ','CHAF']
color_codes = ['m','b']
# ============================================================================================
st = study('Olfacto')
path_phase = path.join(st.path, 'feature/1_Phase_Encoding_EpiPerf_LowHigh/')
save_path = path.join(st.path, 'classified/1_Classif_Phase_Encoding_EpiPerf_'+conds[0].capitalize()+conds[1].capitalize()+'_1000perm/')
# ============================================================================================

for su, step in product(subjects,steps):
    phase_list = []
    #=========================== Load phase files (nfreq, nelec, nwin, ntrial) =================================    
    mat1 = np.load(path.join(path_phase, su+'_'+step+'_'+conds[0]+'_bipo_sel_physFT_phase.npz'))
    names, channels, freq_names = mat1['Mai_RL'], mat1['channels'],mat1['fname']
    sWidth, sStep = 358, 51
    npts = mat1['xPhase'][:,:,1024:2816,:].shape[2]
    time = np.arange(-512,npts-512,52)/512
    print(mat1['xPhase'].shape, mat1['sPhase'][:,:,17:52,:].shape,len(time))
    phase_list.append(mat1['sPhase'][:,:,17:52,:])
    print(phase)
    nelecs = mat1['sPhase'].shape[0]
    mat2name = path.join(path_phase, su+'_'+step+'_'+conds[1]+'_bipo_sel_physFT_phase.npz')
    if path.isfile(mat2name) == True:
        phase_list.append(np.load(mat2name)['sPhase'][:,:,17:52,:])
        print (su, 'phase shape: ', [phase.shape for phase in phase_list])
    
        # =========================== Select phase for 1 elec 1 freq =================================                 
        for elec_num in range(nelecs):
            for freq in range(nfreq):
                elec, elec_label, freq_name = channels[elec_num], names[elec_num], freq_names[freq]
                if path.exists(save_path+'/'+str(freq)+'_'+freq_name+'/auc/'+su +'_auc_'+conds[0]+'_'+conds[1]+'_'+str(elec_label)+'_('+str(elec_num)+').npy'):
                    print(elec_num,' already computed')
                else:
                    print('--» processing',su, 'elec', elec_num,'/',nelecs, 'freq',freq)
                    phase_data_elec = []
                    for i,phase in enumerate(phase_list): #phase file (elecs, freqs, wins,trials))
                        phase_data_elec.append(phase[elec_num,freq].swapaxes(0,1))
                    nwin = phase.shape[1]
                    print ('elec ', elec, 'elec_label ', elec_label)

            # =============================  Classification Computation ============================================================           
                    # create a data matrix, concatenate along the trial dimension
                    x = np.concatenate(phase_data_elec, axis=0)
                    x = np.rad2deg(x)
                    print ('Size of the concatenated data: ', x.shape, 'Number time windows : ', x.shape[1])
                    y = np.hstack([np.array([i]*len(phase)) for i, phase in enumerate(phase_data_elec)])
                    print ('Size of label for classif: ', len(y))

                    auc = np.array([])
                    for t in range(x.shape[1]):
                        X = x[:,t]
                        X = X.reshape(-1, 1)
                        score_rep = []
                        for i in range(10):
                            k = 5
                            if su == 'CHAF':
                                k = 3
                            skf = SKFold(n_splits=k, random_state=None, shuffle=True)
                            skf.get_n_splits(X, y)
                            score_cv = []
                            for train_index, test_index in skf.split(X, y):
                                clf = LDA()
                                X_train, X_test = X[train_index], X[test_index]
                                y_train, y_test = y[train_index], y[test_index]
                                clf.fit(X=X_train, y=y_train)
                                y_pred = clf.predict(X_test)
                                score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                            score_rep.append(np.mean(score_cv))
                        score_rep = np.asarray(score_rep).reshape(1,len(score_rep))
                        auc = np.vstack((auc, score_rep)) if np.size(auc) else score_rep
                    auc = np.swapaxes(auc,0,1)

                    perm_scores = np.array([])
                    for t in range(x.shape[1]):
                        X = x[:,t]
                        X = X.reshape(-1, 1)
                        perm_rep = []
                        for perm in range(nperm):
                            y_perm = y[permutation(len(y))]
                            score_cv = []
                            for train_index, test_index in skf.split(X, y_perm):
                                clf = LDA()
                                X_train, X_test = X[train_index], X[test_index]
                                y_train, y_test = y_perm[train_index], y_perm[test_index]
                                clf.fit(X=X_train, y=y_train)
                                y_pred = clf.predict(X_test)
                                score_cv.append(roc_auc_score(y_test,y_pred,average='weighted'))
                            perm_rep.append(np.mean(score_cv))
                        perm_rep = np.asarray(perm_rep).reshape(1,len(perm_rep))
                        perm_scores = np.vstack((perm_scores, perm_rep)) if np.size(perm_scores) else perm_rep
                    perm_scores = np.swapaxes(perm_scores,0,1)
                    th_0_05_perm = perm_pvalue2level(perm_scores, p=0.05, maxst=True)
                    th_0_01_perm = perm_pvalue2level(perm_scores, p=0.01, maxst=True)
                    print('th_perm : ', th_0_05_perm[0], th_0_01_perm[0], 'auc_mean', auc.mean())

            # ============================== PLOT phase ANALYSIS + STATS & DECODING ACCURACY ===================================================
                    # plot and figure parameters
                    xfmt = ScalarFormatter(useMathText=True)
                    xfmt.set_powerlimits((0,3))
                    fig = plt.figure(1,figsize=(7,7))
                    title = 'Phase-DA for '+su+' '+conds[0].capitalize()+' '+conds[1].capitalize()+' '+str(elec)+' '+str(elec_label)+' ('+str(elec_num)+')'
                    fig.suptitle(title, fontsize=12)

                    # Plot the phase + STATS
                    plt.subplot(211)      
                    BorderPlot(time, x, y=y, kind='sem', alpha=0.2, color=color_codes,linewidth=2, 
                               ncol=1, xlabel='Time (s)',ylabel = r'phase', legend=conds)
                    rmaxis(plt.gca(), ['right', 'top'])
                    plt.legend(loc=0, handletextpad=0.1, frameon=False)
                    plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))

                    # Plot DA for the phase
                    plt.subplot(212)
                    BorderPlot(time, auc, color='b', kind='sd',xlabel='Time (s)', ylim=[0.4,1.], ylabel='Decoding accuracy (%)',linewidth=2, alpha=0.3)
                    rmaxis(plt.gca(), ['right', 'top'])
                    plt.gca().yaxis.set_major_locator(MaxNLocator(3,integer=True))
                    plt.plot(time, th_0_05_perm*np.ones(len(time)), '--', color='r', linewidth=2)
                    #plt.plot(times_plot, th_0_01_perm*np.ones(len(times_plot)), '--', color='orange', linewidth=2)

                    #Filenames to save
                    name_auc = (save_path+'/'+str(freq)+'_'+freq_name+'/auc/'+su +'_auc_'+conds[0]+'_'+conds[1]+'_'+str(elec_label)+'_('+str(elec_num)+').npy')
                    name_perm = (save_path+'/'+str(freq)+'_'+freq_name+'/auc/'+su +'_perm_'+str(elec_label)+'_('+str(elec_num)+').npy')
                    plot_name = (save_path+'/'+str(freq)+'_'+freq_name+'/fig/'+su +'_Phase_'+conds[0]+'_'+conds[1]+'_'+str(elec)+'_'+str(elec_label)+'_('+str(elec_num)+').png')

                    # Criteria to be significant
                    #auc_pvals = np.ravel(auc_pvals)
                    #underp = np.where(auc_pvals < 0.05)[0]
                    #pvsplit = np.split(underp, np.where(np.diff(underp) != 1)[0]+1)
                    #signif = [True for k in pvsplit if len(k) >= 3]
                    #if len(signif) >=1:
                        #plot_sig = (save_path+'/'+str(freq)+'_'+freq_name+'/signif/'+su +'_Phase_'+conds[0].capitalize()+'_'+conds[1].capitalize()+'_'+str(elec)+'_'+str(elec_label)+'_('+str(elec_num)+').png')            
                        #plt.savefig(plot_sig, dpi=300, bbox_inches='tight')

                    #Save plots
                    np.save(name_auc, auc)
                    np.save(name_perm, perm_scores)
                    plt.savefig(plot_name, dpi=300, bbox_inches='tight')
                    plt.clf()
                    plt.close() 
                    del X, auc, phase_data_elec
        del phase_list

    else: 
        print(su,'pas de condition ',conds[1].capitalize())

-> Olfacto loaded
(65, 4, 3584, 31) (65, 4, 35, 31) 35
<class 'brainpipe.feat.basics.phase'>
SEMC phase shape:  [(65, 4, 35, 31), (65, 4, 35, 9)]
0  already computed
0  already computed
0  already computed
0  already computed
1  already computed
1  already computed
1  already computed
1  already computed
2  already computed
2  already computed
2  already computed
2  already computed
3  already computed
3  already computed
3  already computed
3  already computed
4  already computed
4  already computed
4  already computed
4  already computed
5  already computed
5  already computed
5  already computed
5  already computed
6  already computed
6  already computed
6  already computed
6  already computed
7  already computed
7  already computed
7  already computed
7  already computed
8  already computed
8  already computed
8  already computed
8  already computed
9  already computed
9  already computed
9  already computed
9  already computed
10  already computed
10  already computed
10  already 